# Data Loading

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys    
path_to_module = '/content/gdrive/MyDrive/SYDE522'
sys.path.append(path_to_module)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip  '/content/gdrive/MyDrive/SYDE522/archive.zip' -d /content/

In [2]:
train_dir='/content/Vegetable Images/train'
val_dir='/content/Vegetable Images/validation'
test_dir='/content/Vegetable Images/test'

# Keras Model --> CNN + Softmax

In [3]:
import os
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen_flow = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
valid_datagen_flow = valid_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
test_datagen_flow = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=1, seed=42, class_mode='categorical')

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(15, activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', metrics=['acc', 'AUC'], optimizer='adam')
model.fit(train_datagen_flow, validation_data=valid_datagen_flow, steps_per_epoch=len(train_datagen_flow), 
          validation_steps=len(valid_datagen_flow), epochs=5, verbose=1)

Epoch 1/5
469/469 [==============================] - 118s 243ms/step - loss: 1.1227 - acc: 0.6243 - auc: 0.9516 - val_loss: 0.5550 - val_acc: 0.8227 - val_auc: 0.9874
Epoch 2/5
469/469 [==============================] - 113s 240ms/step - loss: 0.4122 - acc: 0.8675 - auc: 0.9912 - val_loss: 0.5022 - val_acc: 0.8443 - val_auc: 0.9864
Epoch 3/5
469/469 [==============================] - 112s 239ms/step - loss: 0.2695 - acc: 0.9165 - auc: 0.9949 - val_loss: 0.3777 - val_acc: 0.8953 - val_auc: 0.9888
Epoch 4/5
469/469 [==============================] - 113s 240ms/step - loss: 0.1902 - acc: 0.9438 - auc: 0.9966 - val_loss: 0.3300 - val_acc: 0.9010 - val_auc: 0.9910
Epoch 5/5
469/469 [==============================] - 113s 240ms/step - loss: 0.1310 - acc: 0.9600 - auc: 0.9978 - val_loss: 0.1977 - val_acc: 0.9473 - val_auc: 0.9961


In [7]:
model.evaluate(test_datagen_flow)

3000/3000 [==============================] - 20s 7ms/step - loss: 0.2101 - acc: 0.9380 - auc: 0.9955


[0.21013502776622772, 0.9380000233650208, 0.9954859018325806]

# Keras Model --> CNN + SVM

In [8]:
import os
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
import seaborn as sns

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen_flow = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
valid_datagen_flow = valid_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
test_datagen_flow = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=1, seed=42, class_mode='categorical')

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [11]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(15, kernel_regularizer=l2(0.001)))
model.add(Activation('linear'))

In [12]:
model.compile(loss='squared_hinge', optimizer='adam', metrics=['accuracy'])
model.fit(train_datagen_flow, validation_data=valid_datagen_flow, steps_per_epoch=len(train_datagen_flow), 
          validation_steps=len(valid_datagen_flow), epochs=5, verbose=1)

Epoch 1/5
469/469 [==============================] - 115s 244ms/step - loss: 0.3070 - accuracy: 0.5427 - val_loss: 0.1188 - val_accuracy: 0.7780
Epoch 2/5
469/469 [==============================] - 113s 240ms/step - loss: 0.0839 - accuracy: 0.8537 - val_loss: 0.0781 - val_accuracy: 0.8713
Epoch 3/5
469/469 [==============================] - 113s 240ms/step - loss: 0.0514 - accuracy: 0.9273 - val_loss: 0.0655 - val_accuracy: 0.8920
Epoch 4/5
469/469 [==============================] - 112s 239ms/step - loss: 0.0353 - accuracy: 0.9588 - val_loss: 0.0390 - val_accuracy: 0.9503
Epoch 5/5
469/469 [==============================] - 113s 240ms/step - loss: 0.0250 - accuracy: 0.9755 - val_loss: 0.0396 - val_accuracy: 0.9443


In [13]:
model.evaluate(test_datagen_flow)

3000/3000 [==============================] - 19s 6ms/step - loss: 0.0417 - accuracy: 0.9397


[0.041656963527202606, 0.9396666884422302]

# Kera Model --> CNN + Softmax & SVM

In [14]:
import os
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
import seaborn as sns

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255,
                    shear_range=0.2,
                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen_flow = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
valid_datagen_flow = valid_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, seed=42, class_mode='categorical')
test_datagen_flow = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=1, seed=42, class_mode='categorical')

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [16]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(15, activation='softmax', kernel_regularizer=l2(0.001)))

In [17]:
model.compile(loss='squared_hinge', optimizer='adam', metrics=['accuracy'])
model.fit(train_datagen_flow, validation_data=valid_datagen_flow, steps_per_epoch=len(train_datagen_flow), 
          validation_steps=len(valid_datagen_flow), epochs=5, verbose=1)

Epoch 1/5
469/469 [==============================] - 113s 240ms/step - loss: 1.0753 - accuracy: 0.4149 - val_loss: 1.0393 - val_accuracy: 0.5787
Epoch 2/5
469/469 [==============================] - 111s 237ms/step - loss: 1.0186 - accuracy: 0.6727 - val_loss: 1.0035 - val_accuracy: 0.7310
Epoch 3/5
469/469 [==============================] - 111s 237ms/step - loss: 1.0026 - accuracy: 0.7399 - val_loss: 0.9862 - val_accuracy: 0.8043
Epoch 4/5
469/469 [==============================] - 111s 237ms/step - loss: 0.9849 - accuracy: 0.8093 - val_loss: 0.9899 - val_accuracy: 0.7910
Epoch 5/5
469/469 [==============================] - 111s 237ms/step - loss: 0.9804 - accuracy: 0.8242 - val_loss: 0.9776 - val_accuracy: 0.8370


In [18]:
model.evaluate(test_datagen_flow)

3000/3000 [==============================] - 18s 6ms/step - loss: 0.9766 - accuracy: 0.8403


[0.9765672087669373, 0.8403333425521851]